## Usage of sclang in sc3nb

You can send commands and receive data directly from the SuperCollider Language

In [1]:
import time
import numpy as np

import sc3nb as scn

In [2]:
sc = scn.startup()

<IPython.core.display.Javascript object>

Starting sclang process...


Done.
Registering OSC /return callback in sclang...
Done.
Loading default SynthDescs
Done.
Booting SuperCollider Server...


Done.


### sclang command execution

To send sc3 commands (i.e. program strings) to the language, either use the functions
- **`cmd()`** normal command sending.
- **`cmdv()`** verbose, i.e. returning string is collected and output to notebook ( alias for `cmd(.., verbose=True)` )
- **`cmdg()`** send program and get and parse the output ( alias for `cmd(.., get_return=True)` )

In [3]:
# sc.cmd(cmdstr, pyvars)
sc.lang.cmd('"hello".postln')  # check jupyter console for output

hello
-> hello


use cmdv If sclang output should be displayed as cell output

In [4]:
sc.lang.cmdv('"sc3nb".postln')  # also check jupyter console for output

sc3nb
-> sc3nb


or use the corresponding Magics in Jupyter

* Jupyter line magics **%sc, %scv, %scg, %scgv**
* Jupyter cell magics **%%sc, %%scv, %%scg, %%scgv**

which wrap the above functions. See examples below

Cell magics can be placed within code just as the function calls as shown here:

In [5]:
%sc x = Synth.new(\default, [\freq, 100])
for p in range(1, 10):  # a bouncing ball
    %sc Synth.new(\s1, [\freq, 200])  // this is sc cell so use sc3 comments instead of #
    time.sleep(1/p)
# NOTE: Windows needs delay between sclang inputs, so use sleep

%sc x.free 

-> Synth('default' : 1000)
-> Synth('s1' : 1001)


-> Synth('s1' : 1000)


-> Synth('s1' : 1001)


-> Synth('s1' : 1002)


-> Synth('s1' : 1003)


-> Synth('s1' : 1004)
-> Synth('s1' : 1005)


-> Synth('s1' : 1006)
-> Synth('s1' : 1007)


-> Synth('default' : 1000)


Use raw python strings for multiline sc3-programs:

In [6]:
sc.lang.cmd(r"""
Routine({
    x = 10.collect{ |i|
        0.2.wait;
        Synth.new(\default, [\freq, 50+(50*i)]);
    };
    1.wait;
    x.do{|e| 
        e.release;
        0.1.wait;};
}).play;
""")

-> a Routine


alternatively, you can use the cell magic:

In [7]:
%%sc
Routine({
    x = 5.collect{ |i|
        0.2.wait;
        Synth.new(\default, [\freq, 50+(50*i)]);
    };
    1.wait;
    x.do{|e| 
        e.release;
        0.1.wait;};
}).play;

-> a Routine


Note that the code is executed in sclang and python is returning directly after sending the command.

### sclang command execution with python variable injection

Python variables can be injected into sc3 commands by using the ^ special: The following examples demonstrates it by setting frequencies by using python variables

In [8]:
for p in range(1, 50):  # a tone ladder
    freq = 50 + p*3
    dur = np.log(p)
    position = np.sign(p-25)
    %sc Synth.new(\s1, [\freq, ^freq, \dur, ^dur, \pan, ^position])
    time.sleep(0.05)

-> Synth('s1' : 1008)
-> Synth('s1' : 1009)
-> Synth('s1' : 1010)
-> Synth('s1' : 1011)


-> Synth('s1' : 1014)
-> Synth('s1' : 1015)
-> Synth('s1' : 1016)
-> Synth('s1' : 1017)


-> Synth('s1' : 1020)
-> Synth('s1' : 1021)
-> Synth('s1' : 1022)
-> Synth('s1' : 1024)


-> Synth('s1' : 1026)
-> Synth('s1' : 1027)
-> Synth('s1' : 1028)
-> Synth('s1' : 1031)


-> Synth('s1' : 1032)
-> Synth('s1' : 1033)
-> Synth('s1' : 1034)
-> Synth('s1' : 1037)


...

-> Synth('s1' : 1053)
-> Synth('s1' : 1054)
-> Synth('s1' : 1056)
-> Synth('s1' : 1057)


-> Synth('s1' : 1058)
-> Synth('s1' : 1059)
-> Synth('s1' : 1061)
-> Synth('s1' : 1062)


-> Synth('s1' : 1063)
-> Synth('s1' : 1064)
-> Synth('s1' : 1065)
-> Synth('s1' : 1066)


-> Synth('s1' : 1067)
-> Synth('s1' : 1068)
-> Synth('s1' : 1069)
-> Synth('s1' : 1070)


-> Synth('s1' : 1071)


This is injection is done with

In [9]:
scn.sclang.convert_to_sc?

In [10]:
python_list = [1,2,3,4]
%scv ^python_list.class

-> Array


In [11]:
complex_py = 1+1j
%scv ^complex_py.class

-> Complex


In [12]:
symbol = r"\\python"
%scv ^symbol.class

-> Symbol


When using the `cmd` | `cmdv` | `cmdg` functions you can also provide a dictionary with variable names as keys and content as values (which can use other python vars or statements)

In [13]:
sc.lang.cmdv("^name1 / ^name2", pyvars={'name1': 9,'name2': 9*2})

-> 0.5


In [14]:
# without providing pyvars, variables are searched in the users namespace.
freq = 5
rate = 6
sc.lang.cmdv("(^freq + 1) * (^rate + 1)")

-> 42


In [15]:
# alternatively via the magic this is done as:
%scv (^freq + 1) * (^rate + 1)

-> 42


### Getting sclang output in python

* To get the output of an sclang snippet into a python variable, use the cmdg function.
* The following example shows how to transfer a synth's nodeID

In [16]:
# start a Synth
sc.lang.cmd(r"""x = Synth.new(\default)""")

-> Synth('default' : 1072)


In [17]:
# get the nodeId to python
nodeID = sc.lang.cmdg("x.nodeID")
print(nodeID)

-> 1072
1072


In [18]:
# use the nodeID to free the Synth via a message to scsynth audio server directly
sc.server.msg("/n_free", nodeID)  

**sc.cmdg(), resp. %scg return integers, floats, strings and lists**
* %scg can be assigned to a python variable within code

In [19]:
a = %scg 1234 + 23452
print(f"returned an {type(a)} of value {a}")

-> 24686
returned an <class 'int'> of value 24686


In [20]:
a = %scg 1234.5.squared
print(f"returned an {type(a)} of value {a}")

-> 1523990.25
returned an <class 'float'> of value 1523990.25


In [21]:
%scg "sonification".scramble

-> sifitioaconn


'sifitioaconn'

be careful when using magics directly after another

In [22]:
%sc ~retval = "sonification".scramble
time.sleep(0.001)  # (in Windows) without pause an Empty might be thrown..
%scg ~retval

-> aiiocnitfnos
-> aiiocnitfnos


'aiiocnitfnos'

(in Windows) it should be prefered to combine your code in one cmd execution.

In [23]:
scramble = %scg ~retval = "sonification".scramble; ~retval ++ "!";
scramble

-> sciatnoniifo!


'sciatnoniifo!'

In [24]:
a = %scg (1,1.1..2)

-> [ 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9 ]


In [25]:
a

[1.0,
 1.100000023841858,
 1.2000000476837158,
 1.2999999523162842,
 1.399999976158142,
 1.5,
 1.600000023841858,
 1.7000000476837158,
 1.7999999523162842,
 1.899999976158142]

In [26]:
a = %scg (1,1.1..2)
print(f"list with length: {len(a)}")
a

-> [ 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9 ]
list with length: 10


[1.0,
 1.100000023841858,
 1.2000000476837158,
 1.2999999523162842,
 1.399999976158142,
 1.5,
 1.600000023841858,
 1.7000000476837158,
 1.7999999523162842,
 1.899999976158142]

Note that floating-point numbers do have a limited precision

In [27]:
[round(num, 6) for num in a]

[1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9]

However they should be close

In [28]:
np.allclose(a, np.arange(1, 2, 0.1))

True

### Cell magics

In [29]:
%sc {SinOsc.ar(MouseX.kr(200,400))}.play  // move mouse horizontally, CMD-. to stop

-> Synth('temp__0' : 1073)


In [30]:
%sc s.scope()

-> a Stethoscope


In [31]:
%sc s.freeAll

-> sc3nb_remote


In [32]:
value = %scg 75-25
print("value = ", value)

-> 50
value =  50


In [33]:
%%sc
{
    x = Synth.new(\s2, [\freq, 100, \num, 1]);
    250.do{|i|
        x.set(\freq, sin(0.2*i.pow(1.5))*100 + 200);
        0.02.wait;
    };
    x.free;
}.fork

-> a Routine


* Try %scv and %%scv for verbose line resp. cells
* Try %scg and %%scg for getter line resp. cells

### Stop synths

In [34]:
synth = "default"
%sc x = Synth.new(^synth)

-> Synth('default' : 1076)


* to stop all playing synths either use CMD-. (in Jupyter Command mode).
* It is a shortcut for the ´free_all´ method of the default server

In [35]:
%sc s.freeAll  // leaves the s.scope running

-> sc3nb_remote


In [36]:
scn.SC.default.server.free_all()

which is also available using our `sc` instance directly

In [37]:
sc.server.free_all()

In [38]:
sc.exit()